In [ ]:
!pip install jcopml

     |████████████████████████████████| 102kB 7.3MB/s 
  Created wheel for jcopml: filename=jcopml-1.1.10-py2.py3-none-any.whl size=33230 sha256=c8dd958728938641dbcd56748e54f9925226e3fe98dc4c69082bd71f531121b4
  Stored in directory: /root/.cache/pip/wheels/82/22/ce/b8b4c3400f788e5050b5b4b4d988884b8ec72b6b8751693914
Successfully built jcopml


In [ ]:
!pip install xlrd

In [ ]:
#import all necessary packages
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt
import scipy.stats as stat
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from jcopml.utils import save_model, load_model
from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Dummy Outage.xlsx to Dummy Outage.xlsx


In [ ]:
df = pd.read_excel(io.BytesIO(uploaded['Dummy Outage.xlsx']))
df.head()

,No.,Outage,Outage Month,Outage Hour,Outage Timing,Outage Day,Outage Days,Outage Date,Year,Re/Patrol,Feeder / Unit,Area,Ops Area,Weather,Cause,Loss,R E M A R K
0,1,2020-01-10 07:25:00,January,07:25:00,Morning,Friday,Weekdays,10,2020,2020-01-10 07:25:00,MINAS POWER PLANT,Minas,PGT South,NaN,NaN,-,"07:25 MGT#5 trip, by alarm Bus Under voltage. ..."
1,2,2020-01-10 07:25:00,January,07:25:00,Morning,Friday,Weekdays,10,2020,2020-01-10 07:25:00,13.8KV MINAS FEEDER #6,Minas,PGT South,Lightning,Animal,75,07:25 Minas ACB fdr#6 T/LO Ctr:207-209 R/F:51@...
2,3,2020-01-10 13:30:00,January,13:30:00,Afternoon,Friday,Weekdays,10,2020,2020-01-10 13:30:00,13.8KV 4D FEEDER #3,Minas,PGT South,Rain,Unidentified,-,4D fdr#3 T/R 1x Ctr:307-308 R/F:51@2@3. Not ch...
3,NaN,2020-01-10 13:30:00,January,13:30:00,Afternoon,Friday,Weekdays,10,2020,2020-05-10 17:03:00,13.8KV 4D FEEDER #3,Minas,PGT South,Rain,Unidentified,-,Checked And Found 4DSB OCB3 Fdr #8 Ctr 307-308...
4,4,2020-01-10 14:00:00,January,14:00:00,Afternoon,Friday,Weekdays,10,2020,2020-01-10 14:00:00,13.8KV DURI COMMUNITY FEEDER #3,Duri,PGT North,Rain and Lightning,Lightning Arrester,-,Duri Community ACB fdr#3 T/R 1x. Not checked y...


In [ ]:
#Check for missing data
missing_data = df.isnull()
missing_data.head()

,No.,Outage,Outage Month,Outage Hour,Outage Timing,Outage Day,Outage Days,Outage Date,Year,Re/Patrol,Feeder / Unit,Area,Ops Area,Weather,Cause,Loss,R E M A R K
0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
missing_data.sum()

No.              735
Outage             0
Outage Month       0
Outage Hour        0
Outage Timing      0
Outage Day         0
Outage Days        0
Outage Date        0
Year               0
Re/Patrol          0
Feeder / Unit      0
Area              24
Ops Area          24
Weather          176
Cause            185
Loss               0
R E M A R K        0
dtype: int64

In [ ]:
#See where the missing data is located
mask = df.isnull().any(axis=1)
df[mask].head()

,No.,Outage,Outage Month,Outage Hour,Outage Timing,Outage Day,Outage Days,Outage Date,Year,Re/Patrol,Feeder / Unit,Area,Ops Area,Weather,Cause,Loss,R E M A R K
0,1,2020-01-10 07:25:00,January,07:25:00,Morning,Friday,Weekdays,10,2020,2020-01-10 07:25:00,MINAS POWER PLANT,Minas,PGT South,NaN,NaN,-,"07:25 MGT#5 trip, by alarm Bus Under voltage. ..."
3,NaN,2020-01-10 13:30:00,January,13:30:00,Afternoon,Friday,Weekdays,10,2020,2020-05-10 17:03:00,13.8KV 4D FEEDER #3,Minas,PGT South,Rain,Unidentified,-,Checked And Found 4DSB OCB3 Fdr #8 Ctr 307-308...
5,NaN,2020-01-10 14:00:00,January,14:00:00,Afternoon,Friday,Weekdays,10,2020,2020-05-10 08:00:00,13.8KV DURI COMMUNITY FEEDER #3,Duri,PGT North,Rain and Lightning,Lightning Arrester,-,Checked & found Duri Community fdr#3 T/R 1x Ct...
8,NaN,2020-02-10 08:41:00,February,08:41:00,Morning,Monday,Weekdays,10,2020,2020-05-10 15:16:00,13.8KV KETIGUL FEEDER#1,Bangko-Balam,PGT North,Fair,Dead End Insulator,22,Checked And Found PKTGL VCB1 Fdr #8 Ctr 243-24...
9,7,2020-02-10 10:40:00,February,10:40:00,Morning,Monday,Weekdays,10,2020,2020-02-10 10:40:00,CENTRAL DURI POWER PLANT,Duri,PGT North,NaN,NaN,-,"S/U CGT#5, then on line the unit at 11:06."


In [ ]:
#Replace missing data that turned to be text object with the word None
df.fillna(value='Unidentified',inplace=True)
df[mask].head()

,No.,Outage,Outage Month,Outage Hour,Outage Timing,Outage Day,Outage Days,Outage Date,Year,Re/Patrol,Feeder / Unit,Area,Ops Area,Weather,Cause,Loss,R E M A R K
0,1,2020-01-10 07:25:00,January,07:25:00,Morning,Friday,Weekdays,10,2020,2020-01-10 07:25:00,MINAS POWER PLANT,Minas,PGT South,Unidentified,Unidentified,-,"07:25 MGT#5 trip, by alarm Bus Under voltage. ..."
3,Unidentified,2020-01-10 13:30:00,January,13:30:00,Afternoon,Friday,Weekdays,10,2020,2020-05-10 17:03:00,13.8KV 4D FEEDER #3,Minas,PGT South,Rain,Unidentified,-,Checked And Found 4DSB OCB3 Fdr #8 Ctr 307-308...
5,Unidentified,2020-01-10 14:00:00,January,14:00:00,Afternoon,Friday,Weekdays,10,2020,2020-05-10 08:00:00,13.8KV DURI COMMUNITY FEEDER #3,Duri,PGT North,Rain and Lightning,Lightning Arrester,-,Checked & found Duri Community fdr#3 T/R 1x Ct...
8,Unidentified,2020-02-10 08:41:00,February,08:41:00,Morning,Monday,Weekdays,10,2020,2020-05-10 15:16:00,13.8KV KETIGUL FEEDER#1,Bangko-Balam,PGT North,Fair,Dead End Insulator,22,Checked And Found PKTGL VCB1 Fdr #8 Ctr 243-24...
9,7,2020-02-10 10:40:00,February,10:40:00,Morning,Monday,Weekdays,10,2020,2020-02-10 10:40:00,CENTRAL DURI POWER PLANT,Duri,PGT North,Unidentified,Unidentified,-,"S/U CGT#5, then on line the unit at 11:06."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   No.            2208 non-null   object        
 1   Outage         2208 non-null   datetime64[ns]
 2   Outage Month   2208 non-null   object        
 3   Outage Hour    2208 non-null   object        
 4   Outage Timing  2208 non-null   object        
 5   Outage Day     2208 non-null   object        
 6   Outage Days    2208 non-null   object        
 7   Outage Date    2208 non-null   int64         
 8   Year           2208 non-null   int64         
 9   Re/Patrol      2208 non-null   object        
 10  Feeder / Unit  2208 non-null   object        
 11  Area           2208 non-null   object        
 12  Ops Area       2208 non-null   object        
 13  Weather        2208 non-null   object        
 14  Cause          2208 non-null   object        
 15  Loss           2208 n

In [ ]:
NLP_df = df[['R E M A R K','Cause']]
NLP_df.head()

,R E M A R K,Cause
0,"07:25 MGT#5 trip, by alarm Bus Under voltage. ...",Unidentified
1,07:25 Minas ACB fdr#6 T/LO Ctr:207-209 R/F:51@...,Animal
2,4D fdr#3 T/R 1x Ctr:307-308 R/F:51@2@3. Not ch...,Unidentified
3,Checked And Found 4DSB OCB3 Fdr #8 Ctr 307-308...,Unidentified
4,Duri Community ACB fdr#3 T/R 1x. Not checked y...,Lightning Arrester


In [ ]:
X = NLP_df.iloc[:,0]
X.head()

0    07:25 MGT#5 trip, by alarm Bus Under voltage. ...
1    07:25 Minas ACB fdr#6 T/LO Ctr:207-209 R/F:51@...
2    4D fdr#3 T/R 1x Ctr:307-308 R/F:51@2@3. Not ch...
3    Checked And Found 4DSB OCB3 Fdr #8 Ctr 307-308...
4    Duri Community ACB fdr#3 T/R 1x. Not checked y...
Name: R E M A R K, dtype: object

In [ ]:
X.shape

(2208,)

In [ ]:
y = NLP_df.iloc[:,1]
y.head()

0           Unidentified
1                 Animal
2           Unidentified
3           Unidentified
4     Lightning Arrester
Name: Cause, dtype: object

In [ ]:
y.shape

(2208,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1766,), (442,), (1766,), (442,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
#helper function fo scikit
from jcopml.tuning import random_search_params as rsp
#scikit optimizer
import skopt
from skopt import gp_minimize
from skopt.space import Real, Integer

In [ ]:
pipeline = Pipeline([('prep',TfidfVectorizer(tokenizer=word_tokenize, stop_words= stopwords.words("english"))),('algo', LogisticRegression(solver='lbfgs', n_jobs=-1, random_state=42))])

model = RandomizedSearchCV(pipeline, rsp.logreg_params, cv=3, n_iter=50, n_jobs=-1, verbose=1, random_state=42)
model.fit(X_train, y_train)
print(model.get_params)
print(model.score(X_train, y_train), model.best_score_, model.score(X_test, y_test))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('prep',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
              

In [ ]:
prediction = pd.DataFrame(X_test)
prediction["true_category"] = y_test
prediction["predicted_category"] = model.predict(X_test)
prediction.head()

,R E M A R K,true_category,predicted_category
2057,10:27 Ketigul Feeder#2 T/Lo 12:15 Checked and ...,Animal,Animal
1094,"Checked and found RCl 41-AW-62 T/R 1x, ctr: 33...",Unidentified,Unidentified
1472,16:07 MGT #10 S/U and then online at 17:00 for...,Unidentified,Unidentified
430,"10:32 Manual open VCB 8D fdr #2, closed at 10:...",Planned Maintenance,Planned Maintenance
949,10:55 It was reported no power at well Petani ...,Animal,Animal


In [ ]:
prediction["losses"] = np.random.rand(len(prediction))
prediction.head()

,R E M A R K,true_category,predicted_category,losses
2057,10:27 Ketigul Feeder#2 T/Lo 12:15 Checked and ...,Animal,Animal,0.280868
1094,"Checked and found RCl 41-AW-62 T/R 1x, ctr: 33...",Unidentified,Unidentified,0.078400
1472,16:07 MGT #10 S/U and then online at 17:00 for...,Unidentified,Unidentified,0.286925
430,"10:32 Manual open VCB 8D fdr #2, closed at 10:...",Planned Maintenance,Planned Maintenance,0.163958
949,10:55 It was reported no power at well Petani ...,Animal,Animal,0.976027


In [ ]:
prediction.groupby("predicted_category").sum()

,losses
predicted_category,
Unidentified,15.777839
Animal,10.365365
Broken Fuse Holder,0.505155
Construction Job,1.208455
Dead End Insulator,8.911266
Emergency,4.000709
Lightning,50.403923
Lightning Arrester,3.730205
Normal Condition,0.590101


In [ ]:
import os
from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from gensim.models import FastText

In [ ]:
#tokenisasi statement
sentences = [word_tokenize(text.lower()) for text in tqdm (X)]
sentences[:200]

[['07:25',
  'mgt',
  '#',
  '5',
  'trip',
  ',',
  'by',
  'alarm',
  'bus',
  'under',
  'voltage',
  '.',
  '08:10',
  's/u',
  'mgt',
  '#',
  '5',
  ',',
  'then',
  'on',
  'line',
  'the',
  'unit',
  'at',
  '08:17',
  '.'],
 ['07:25',
  'minas',
  'acb',
  'fdr',
  '#',
  '6',
  't/lo',
  'ctr:207-209',
  'r/f:51',
  '@',
  '2',
  '@',
  '3',
  '.',
  '08:55',
  'patrolled',
  'the',
  'line',
  '&',
  'found',
  'dead',
  'squirrel',
  'on',
  'top',
  'well',
  '7e-56',
  '.',
  '09:30',
  'blocked',
  '&',
  'closed',
  'minas',
  'acb',
  'fdr',
  '#',
  '6',
  ',',
  'then',
  'release',
  'trhe',
  'block',
  '.',
  'cause',
  ':',
  'animal',
  '(',
  'squirrel',
  ')',
  '.'],
 ['4d',
  'fdr',
  '#',
  '3',
  't/r',
  '1x',
  'ctr:307-308',
  'r/f:51',
  '@',
  '2',
  '@',
  '3.',
  'not',
  'checked',
  'yet',
  '.'],
 ['checked',
  'and',
  'found',
  '4dsb',
  'ocb3',
  'fdr',
  '#',
  '8',
  'ctr',
  '307-308',
  ',',
  'r/f',
  '51',
  'patrolled',
  'the',
  'li

In [ ]:
model = FastText(sentences, size=128, window=200, min_count=3, workers = 4, iter = 1000, sg=0, hs=0)

In [ ]:
w2v = model.wv

In [ ]:
def norm_sent_vector(sentence, w2v_model):
  vecs = [w2v_model[word.lower()] for word in word_tokenize(sentence)]
  norm_vecs = [vec/np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec > 0)]
  sent_vec = np.mean(vecs, axis=0)

  return sent_vec

In [ ]:
vecs = [norm_sent_vector(sentence, w2v) for sentence in X]
vecs = np.array(vecs)
vecs

KeyError: ignored